In [1]:
import numpy as np
import matplotlib.pyplot as plt
import thomson
import astropy.units as u
from astropy.constants import m_e, m_p
from scipy.special import wofz

Validate non-Maxwellian function

These functions allow us to generate arrays corresponding to Maxwellians with different temperatures and drift velocities.

In [ ]:
def maxwellian_e(v, T, v_d):
    v_unitless = v
    T_K = T * 11605
    vth = np.sqrt(2 * 1.5156e7 * T_K)
    exp_term = -(v_unitless - v_d) ** 2 / (vth ** 2)
    return np.exp(exp_term) / np.sqrt(np.pi * vth ** 2)

def maxwellian_H(v, T, v_d):
    v_unitless = v
    T_K = T * 11605
    vth = np.sqrt(2 * 8.2544e3 * T_K)
    exp_term = -(v_unitless - v_d) ** 2 / (vth ** 2)
    return np.exp(exp_term) / np.sqrt(np.pi * vth ** 2)

In [ ]:
def maxwellian_e(v, T, v_d):
    v_unitless = v
    T_K = T * 11605
    vth = np.sqrt(2 * 1.5156e7 * T_K)
    exp_term = -(v_unitless - v_d) ** 2 / (vth ** 2)
    return np.exp(exp_term) / np.sqrt(np.pi * vth ** 2)

def maxwellian_D(v, T, v_d):
    v_unitless = v
    T_K = T * 11605
    vth = np.sqrt(2 * 8.2544e3 / 2.014 * T_K)
    exp_term = -(v_unitless - v_d) ** 2 / (vth ** 2)
    return np.exp(exp_term) / np.sqrt(np.pi * vth ** 2)

def maxwellian_C(v, T, v_d):
    v_unitless = v
    T_K = T * 11605
    vth = np.sqrt(2 * 8.2544e3 / 12 * T_K)
    exp_term = -(v_unitless - v_d) ** 2 / (vth ** 2)
    return np.exp(exp_term) / np.sqrt(np.pi * vth ** 2)

Now we define the electron and ion distributions. Note that everything is dimensionless now, it's more convenient to put the units on in the function call due to how astropy handles units.

In [1]:
ve = np.linspace(-2e7, 2e7, 1000)
vi = np.linspace(-1e6, 1e6, 1000)

Te = 100
TD = 200
TC = 100

ue = 0
uD = 0
uC = 4e5

n = 1e19

fe = maxwellian_e(ve, Te, 0)
fD = maxwellian_D(vi, TD, 0)
fC = maxwellian_C(vi, TC, uC)

NameError: name 'np' is not defined

In [ ]:
fig, ax = plt.subplots(ncols = 3, figsize = (12, 4))
ax[0].semilogy(ve, fe)
ax[0].set_title("Normalized electron distribution")
ax[0].set_xlabel("v [m/s]")

ax[1].semilogy(vi, fD)
ax[1].set_title("Normalized deuteron distribution")
ax[1].set_xlabel("v [m/s]")

ax[2].semilogy(vi, fC)
ax[2].set_title("Normalized carbon distribution")
ax[2].set_xlabel("v [m/s]")

Define the Thomson parameters:

In [ ]:
probe_wavelength = 532
epw_wavelengths = np.linspace(probe_wavelength-50, probe_wavelength+50, 200)

theta = 63 * np.pi/180

probe_vec = np.array([0, np.sin(theta), np.cos(theta)])
scatter_vec = np.array([0, np.sin(theta), -np.cos(theta)])

notch = np.array([probe_wavelength-10, probe_wavelength+10])

In [ ]:
chiE = thomson.chi(
    wavelengths = epw_wavelengths * u.nm, 
    probe_wavelength = probe_wavelength * u.nm, 
    f_k = fe * u.s / u.m, 
    v_k = ve * u.m / u.s, 
    n = n * u.cm**(-3), 
    probe_vec=probe_vec, 
    scatter_vec = scatter_vec,
    z = 1,
    mass = m_e)

chiE = thomson.chi_lite(
    wavelengths=epw_wavelengths*1e-9,
    probe_wavelength=probe_wavelength*1e-9,
    f_k = fe,
    v_k = ve,
    n = n*1e6,
    probe_vec=probe_vec,
    scatter_vec=scatter_vec,
    z=1,
    mass=m_e.value
)

In [ ]:
m_e.value

In [ ]:
wofz(1)

In [ ]:
plt.plot(epw_wavelengths, np.real(chiE))
plt.plot(epw_wavelengths, np.imag(chiE))

In [ ]:
alpha, epw_Skw = thomson.spectral_density(
    wavelengths = epw_wavelengths * u.nm,
    probe_wavelength = probe_wavelength * u.nm,
    n = n * u.cm**(-3),
    fe_k = np.array([fe]) * u.s / u.m,
    ve_k = np.array([ve]) * u.m / u.s,
    fi_k = np.array([fD]) * u.s / u.m,
    vi_k = np.array([vi]) * u.m / u.s,
    #ifract = [0.2, 0.8],
    ions = ['D 1+'],
    probe_vec=probe_vec,
    scatter_vec=scatter_vec,
    #notch = notch * u.nm
)
print(alpha)

alpha, epw_Skw_maxwellian = thomson.spectral_density_maxwellian(
    wavelengths = epw_wavelengths * u.nm,
    probe_wavelength = probe_wavelength * u.nm,
    n = n * u.cm**(-3),
    T_e = Te * u.eV,
    T_i = np.array([TD])*u.eV,
    #ifract = [0.2],
    ions = ['D 1+'],
    probe_vec=probe_vec,
    scatter_vec=scatter_vec,
    
)
print(alpha)

In [ ]:
plt.plot(epw_wavelengths, epw_Skw, label='')

plt.plot(epw_wavelengths, epw_Skw_maxwellian, label='')